# Turn-by-turn BPM data in the SNS ring

In [ ]:
import sys
from pprint import pprint

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import proplot as plot

sys.path.append('/Users/46h/Research/accphys/')
from tools import animation as myanim
from tools import plotting as myplt

plot.rc['grid.alpha'] = 0.04
plot.rc['axes.grid'] = False
plot.rc['cycle'] = 'default'
plot.rc['savefig.dpi'] = 'figure'
plot.rc['animation.html'] = 'jshtml'

Each BPM records data for 1059 turns (the number of stored turns in the machine was much less). This was repeated for 50 minipulses.

In [ ]:
df = pd.read_table('ring_tbt.dat', sep=',')
df

Average over the minipulses.

In [ ]:
n_turns = df['turn'].max()
columns = df.columns
df = [df.loc[df['turn'] == i].mean().values for i in range(n_turns)]
df = pd.DataFrame(df, columns=columns)
df = df.drop(columns=['pulse', 'turn'])
df

Find the number of stored turns and cut off all data beyond this turn.

In [ ]:
n_stored_turns = len(df)
for i in reversed(range(len(df))):
    if np.any(df.iloc[i]):
        n_stored_turns = i + 1
        break
df = df.iloc[:n_stored_turns]
df

Learn the BPM names.

In [ ]:
bpm_names = []
for col in df.columns:
    if 'BPM' in col:
        name = None
        if 'xAvg' in col:
            name = col.split(':xAvg')[0]
        elif 'yAvg' in col:
            name = col.split(':yAvg')[0]
        if name and name not in bpm_names:
            bpm_names.append(name)

Load the BPM positions.

In [ ]:
bpm_positions_dict = dict()
file = open('ring_bpm_positions.dat', 'r')
for line in file:
    name, position = line.split(' ')
    position = float(position)
    bpm_positions_dict[name] = position
file.close()

Make sure they are sorted by position. They should be already.

In [ ]:
sorted_name_position_list = sorted(bpm_positions_dict.items(), key=lambda item: item[1])
bpm_names, bpm_positions = zip(*sorted_name_position_list)

Plot example trajectory.

In [ ]:
fig, axes = plot.subplots(nrows=2, figsize=(8, 3.5), spany=False)
for bpm_name in bpm_names[:4]:
    col_xavg = bpm_name + ':xAvg'
    col_yavg = bpm_name + ':yAvg'
    df[col_xavg].plot(ax=axes[0], marker='.', ms=4, label=bpm_name)
    df[col_yavg].plot(ax=axes[1], marker='.', ms=4, label=bpm_name)
axes[0].format(ylabel='x [mm]', title='TBT data for selected BPMS')
axes[1].format(ylabel='y [mm]', xlabel='Turn number')
axes[0].legend(ncol=1, loc=(1.01, 0));

## Phase space 

Try to plot the phase space just before a BPM. I guess I'm assuming that there are no quads between the BPMs, and that the BPMs are located at the quad centers.

In [ ]:
def bpm_phase_space_coords(bpm_name, slope='before'):        
    x_avg = df[bpm_name + ':xAvg']
    y_avg = df[bpm_name + ':yAvg']
    i = bpm_names.index(bpm_name)
    if slope == 'before':
        if i == 0:
            raise ValueError('Cannot plot phase space before first BPM')
        name_before = bpm_names[i - 1]
        delta_x = x_avg - df[name_before + ':xAvg']
        delta_y = y_avg - df[name_before + ':yAvg']
        delta_s = bpm_positions[i] - bpm_positions[i - 1]
    elif slope == 'after':
        if i == len(bpm_names):
            raise ValueError('Cannot plot phase space before first BPM')
        name_after = bpm_names[i + 1]
        delta_x = df[name_after + ':xAvg'] - x_avg
        delta_y = df[name_after + ':yAvg'] - y_avg
        delta_s = bpm_positions[i + 1] - bpm_positions[i]
    else:
        raise ValueError("`slope` must be in {'before', 'after'}")
    xp_avg = delta_x / delta_s
    yp_avg = delta_y / delta_s
    coords = np.vstack([x_avg, xp_avg, y_avg, yp_avg]).T
    return coords

In [ ]:
colors = ['red8', 'blue8']

for bpm_name, bpm_pos in zip(bpm_names, bpm_positions):
    X_before, X_after = None, None
    if bpm_name != bpm_names[0]:
        X_before = bpm_phase_space_coords(bpm_name, slope='before')
    if bpm_name != bpm_names[-1]:
        X_after = bpm_phase_space_coords(bpm_name, slope='after')
    coords = [X_before, X_after]
    if X_after is not None and X_before is None:
        colors = list(reversed(colors))
        coords = list(reversed(coords))
                
    axes = myplt.corner(coords[0], diag_kind=None, s=20, c='red8')
    if coords[1] is not None:
        for i in range(3):
            for j in range(i + 1):
                ax = axes[i, j]
                ax.scatter(coords[1][:, j], coords[1][:, i + 1], s=2, c='blue8')
    axes[0, 1].annotate(bpm_name, xy=(0, 0.425), xycoords='axes fraction', fontsize=8.75)
    lines = [Line2D([0], [0], color=colors[0]), 
             Line2D([0], [0], color=colors[1])]
    axes[0, 0].legend(lines, ['before', 'after'], loc=(1.18, 0), 
                      handlelength=1, framealpha=0)
    plt.show()

In [ ]:
coords_before, coords_after = [], []
for bpm_name, bpm_position in zip(bpm_names[1:-1], bpm_positions[1:-1]):
    coords_before.append(bpm_phase_space_coords(bpm_name, slope='before'))
    coords_after.append(bpm_phase_space_coords(bpm_name, slope='after'))

In [ ]:
anim = myanim.corner(coords_before, diag_kind=None, limits=None, pad=0, s=5,
                     c='red8', text_fmt='{}', text_vals=bpm_names[1:-1])
anim

In [ ]:
anim = myanim.corner(coords_after, diag_kind=None, limits=None, pad=0, s=5,
                     c='blue8', text_fmt='{}', text_vals=bpm_names[1:-1])
anim